In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import helper_functions

In [2]:
path = 'data_folder/'

In [3]:
enrollment_fact = pd.read_csv(path +'enrollment_fact.txt', sep='\t')  # RELACIÓN ENTRE CURSOS, PERIODOS Y ESTUDIANTES
course_dim = pd.read_csv(path +"course_dim.txt", sep='\t') # LISTA DE CURSOS EN GENERAL (ACADEMICOS Y EXTRA-ACADEMICOS)
user_dim = pd.read_csv(path +'user_dim.txt', sep='\t') # LISTA DE ESTUDIANTES
enrollment_term_dim = pd.read_csv(path +'enrollment_term_dim.txt', sep='\t') # LISTA DE PERIODOS 
pseudonym_dim = pd.read_csv(path +'pseudonym_dim.txt', sep='\t') # LISTA DE MATRICULAS
ecoa = pd.read_csv(path + 'GRUPO_ECOA.csv') # CALIFICACIONES DE ECOA
alumno_df = pd.read_table(path +'DET_IND_ALUMNO.tsv') # RELACIÓN ESTUDIANTES CON ECOA Y MATERIAS
cursos_life = pd.read_table(path +'DWH_MATERIAS_EXTRA_ACADEMICAS.tsv') # LISTA DE MATERIAS EXTRA ACADEMICAS
materias_life = cursos_life.CLAVE_MATERIA.unique() # LISTA DE MATERIAS EXTRA ACADEMICAS

In [4]:
enrollment_fact = helper_functions.int_to_str(enrollment_fact)
course_dim = helper_functions.int_to_str(course_dim)
user_dim = helper_functions.int_to_str(user_dim)
enrollment_term_dim = helper_functions.int_to_str(enrollment_term_dim)
pseudonym_dim = helper_functions.int_to_str(pseudonym_dim)
alumno_df = helper_functions.int_to_str(alumno_df)
cursos_life = helper_functions.int_to_str(cursos_life)

In [7]:
# FILTRADO Y MODIFICACIÓN A COURSE_DIM

course_dim.code = course_dim.code.apply(lambda x : x[:x.find('.')]) # Obtener códigos generales de las materias
course_dim = course_dim[course_dim.code.isin(materias_life)] #HACER FILTRADO DE LAS MATERIAS QUE SOLO SON LIFE
course_dim.rename(columns={'id':'course_id'}, inplace = True)
course_dim = course_dim.loc[:, ['course_id','enrollment_term_id', 'name', 'code', 'start_at']] 

In [8]:
# FILTRADO Y MODIFICACIÓN A enrollment_fact

enrollment_fact = enrollment_fact.loc[:, ['enrollment_id', 'user_id', 'course_id', 'enrollment_term_id']]
enrollment_fact

,enrollment_id,user_id,course_id,enrollment_term_id
0,143280000000018596,103729482685559320,143280000000000874,143280000000000007
1,143280000000017210,369986378671339382,143280000000000874,143280000000000007
2,143280000000017212,571665427533582257,143280000000000874,143280000000000007
3,143280000000017216,-532628700693452303,143280000000000874,143280000000000007
4,143280000000017218,-516765647756560062,143280000000000874,143280000000000007
...,...,...,...,...
4627970,143280000003710540,-388274441878071802,143280000000204142,143280000000000141
4627971,143280000003592898,311511555886490383,143280000000204342,143280000000000141
4627972,143280000003598117,41939860565794830,143280000000204342,143280000000000141
4627973,143280000003665067,-235574164982587329,143280000000204342,143280000000000141


In [9]:
# FILTRADO Y MODIFICACIÓN A user_dim

user_dim.rename(columns={'id':'user_id', 'name':'user_name'}, inplace = True)
user_dim = user_dim.loc[:, ['user_id', 'user_name', 'time_zone']]

In [10]:
# FILTRADO Y MODIFICACIÓN A user_dim

pseudonym_dim.rename(columns={'unique_name':'matricula'}, inplace=True)
pseudonym_dim = pseudonym_dim.loc[:, ['id', 'user_id', 'matricula']]

In [11]:

tabla = pd.merge(enrollment_fact, course_dim, left_on=['course_id', 'enrollment_term_id' ], right_on=['course_id', 'enrollment_term_id'])
tabla2 = pd.merge(tabla, user_dim, left_on=['user_id'], right_on=['user_id'])
tabla3 = pd.merge(tabla2, pseudonym_dim, left_on=['user_id'], right_on=['user_id'])
tabla3['start_at'] = tabla3['start_at'].apply(helper_functions.ddate)

In [12]:
ecoa.CLAVE_EJER_ACAD_GRUPO = ecoa.CLAVE_EJER_ACAD_GRUPO.astype('str')

In [15]:
final = pd.DataFrame(columns=['matricula', 'course_code', 'course_name', 'ratings'])

for matricula in tqdm(tabla3.matricula.unique()[:10]):

    df1 = tabla3[tabla3.matricula == matricula].loc[:, ['matricula', 'user_name', 'start_at', 'code']]
    df2 = df1.merge(cursos_life, left_on=['code', 'start_at'], right_on=['CLAVE_MATERIA', 'CLAVE_EJERCICIO_ACADEMICO'])
    df3 = df2.merge(alumno_df, left_on = ['CLAVE_CAMPUS', 'CLAVE_EJERCICIO_ACADEMICO', 'matricula'], 
                            right_on = ['CLAVE_CAMPUS', 'CLAVE_EJERCICIO_ACADEMICO', 'MATRICULA'])

    df4 = df3.merge(ecoa, left_on = ['CLAVE_CAMPUS', 'CLAVE_EJERCICIO_ACADEMICO', 'CLAVE_MATERIA'],
                            right_on = ['CLAVE_CAMPUS_GRUPO', 'CLAVE_EJER_ACAD_GRUPO', 'CLAVE_MATERIA'])

    for c in df3.code.unique():
        p = helper_functions.promedio_p(df4[(df4['code']==c)]['EVAL'])
        df3.loc[(df3['code']==c), 'EVAL'] = int(p)

    df3.rename(columns={'EVAL':'ratings'}, inplace= True)
    final = pd.concat([final, df3])

final = final.drop_duplicates(subset= ['matricula', 'code', 'course_name']).reset_index()
final.loc[:, ['matricula', 'user_name', 'code', 'NOMBRE_MATERIA_CORTO', 'CLAVE_EJERCICIO_ACADEMICO', 'ratings']]

100%|██████████| 10/10 [00:09<00:00,  1.05it/s]


,matricula,user_name,code,NOMBRE_MATERIA_CORTO,CLAVE_EJERCICIO_ACADEMICO,ratings
0,A01411319,María Fernanda Lee García,XAFG3001,Acond físico en gimnasio,202011,8.0
1,A01411319,María Fernanda Lee García,XTOC4001,Sel tocho bandera fem mayor,202011,8.0
2,A01411242,Berenice Pinzón Alarcón,XAFG3002,Acond físico general,202113,5.0
3,A01411235,Erandy Rodríguez Revuelta,XTOC4002,Sel tocho bandera var mayor,202113,4.0
4,A01411991,Paulina Yuriel Moreno Alvarado,KLID3002,Grupos estudiantiles,202011,10.0
5,A01411991,Paulina Yuriel Moreno Alvarado,XTOC4001,Sel tocho bandera fem mayor,202011,8.0
6,A01411672,Ana Gabriela Elver Villegas,XTOC4001,Sel tocho bandera fem mayor,202013,6.0
7,A01411672,Ana Gabriela Elver Villegas,KLID3002,Grupos estudiantiles,202011,10.0
8,A01411672,Ana Gabriela Elver Villegas,YDCU3015,Danza urbana,202011,5.0
9,A01411223,Kimberly Del Ángel Urrutia,KLID3002,Grupos estudiantiles,202013,7.0


In [ ]:
tabla3.sort_values(by='code')

,enrollment_id,user_id,course_id,enrollment_term_id,name,code,start_at,user_name,time_zone,id,matricula
97088,143280000001077687,142147613297030634,143280000000048783,143280000000000012,El arte de prepararte para una carrera (Gpo 2),KCVC3001,02,Raymundo Romero Arenas,Monterrey,143280000000018010,A00570654
268024,143280000000301633,-564222488209049362,143280000000013247,143280000000000002,El arte de prepararte para una carrera (Gpo 4),KCVC3001,201913,Mariely García Moyano Salas,Monterrey,143280000000060230,A01551013
215015,143280000000352747,-100279378308455924,143280000000013247,143280000000000002,El arte de prepararte para una carrera (Gpo 4),KCVC3001,201913,Camila Cabrera Arias,Monterrey,143280000000267510,L03519405
147375,143280000001798390,107383447722882706,143280000000079599,143280000000000126,El arte de prepararte para una carrera (Gpo 2),KCVC3001,202013,Angela Perez Mendoza,Monterrey,143280000000060310,A01703255
287965,143280000001081374,-120505303660156172,143280000000048783,143280000000000012,El arte de prepararte para una carrera (Gpo 2),KCVC3001,02,Andrea Paulina Quezada Corona,Mexico City,143280000000005901,A01701953
...,...,...,...,...,...,...,...,...,...,...,...
113198,143280000001990261,94399573077999840,143280000000071747,143280000000000126,Pintura con acrílico y óleo (Gpo 11),YVPI3003,202013,Norma Patricia Cervantes Flores,Monterrey,143280000000034667,A00823857
249223,143280000004451715,-414977623553672083,143280000000187549,143280000000000141,Pintura con acrílico y óleo (Gpo 3),YVPI3003,09,Lissette Anaid López Buda,Monterrey,143280000000053970,A01654621
97969,143280000002141575,-464889418235717780,143280000000074601,143280000000000126,Pintura con acrílico y óleo (Gpo 11),YVPI3003,202013,Mayra Landin Domínguez,Monterrey,143280000000042358,A01701431
44730,143280000003889977,-527139916413646298,143280000000187539,143280000000000141,Pintura con acrílico y óleo (Gpo 1),YVPI3003,202113,Fátima Galicia Rueda,Monterrey,143280000000167589,A01707082
